In [8]:
# Authors: The MNE-Python contributors.
# License: BSD-3-Clause
# Copyright the MNE-Python contributors.

import base64
import glob
import os
import pickle
import re
import shutil
from io import BytesIO
from pathlib import Path

import numpy as np
import pytest
from matplotlib import pyplot as plt

from mne import (
    Epochs,
    create_info,
    pick_channels_cov,
    read_cov,
    read_events,
    read_evokeds,
)
from mne._fiff.write import DATE_NONE
from mne.datasets import testing
from mne.epochs import make_metadata
from mne.io import RawArray, read_info, read_raw_fif
from mne.preprocessing import ICA
from mne.report import Report, _ReportScraper, open_report, report
from mne.report import report as report_mod
from mne.report.report import (
    _ALLOWED_IMAGE_FORMATS,
    CONTENT_ORDER,
)
from mne.utils import Bunch, _record_warnings
from mne.utils._testing import assert_object_equal
from mne.viz import plot_alignment

data_dir = testing.data_path(download=True)
subjects_dir = data_dir / "subjects"
sample_meg_dir = data_dir / "MEG" / "sample"
raw_fname = sample_meg_dir / "sample_audvis_trunc_raw.fif"
ms_fname = data_dir / "SSS" / "test_move_anon_raw.fif"
events_fname = sample_meg_dir / "sample_audvis_trunc_raw-eve.fif"
evoked_fname = sample_meg_dir / "sample_audvis_trunc-ave.fif"
cov_fname = sample_meg_dir / "sample_audvis_trunc-cov.fif"
ecg_proj_fname = sample_meg_dir / "sample_audvis_ecg-proj.fif"
eog_proj_fname = sample_meg_dir / "sample_audvis_eog-proj.fif"
fwd_fname = sample_meg_dir / "sample_audvis_trunc-meg-eeg-oct-6-fwd.fif"
trans_fname = sample_meg_dir / "sample_audvis_trunc-trans.fif"
inv_fname = sample_meg_dir / "sample_audvis_trunc-meg-eeg-oct-6-meg-inv.fif"
stc_fname = sample_meg_dir / "sample_audvis_trunc-meg"
mri_fname = subjects_dir / "sample" / "mri" / "T1.mgz"
nirs_fname = (
    data_dir / "SNIRF" / "NIRx" / "NIRSport2" / "1.0.3" / "2021-05-05_001.snirf"
)
stc_plot_kwargs = dict(  # for speed
    smoothing_steps=1, size=(300, 300), views="lat", hemi="lh"
)
topomap_kwargs = dict(res=8, contours=0, sensors=False)



Dataset testing version 0.152 out of date, latest version is 0.156


0.00B [00:00, ?B/s]     
Untarring contents of 'C:\Users\victor.ferat\mne_data\mne-testing-data-0.156.tar.gz' to 'C:\Users\victor.ferat\mne_data'


Download complete in 05m25s (1046.5 MB)


In [10]:
import mne

fwd = mne.read_forward_solution(fwd_fname)
if fwd['coord_frame'] == 'head'
src = fwd["src"]
src_

Reading forward solution from C:\Users\victor.ferat\mne_data\MNE-testing-data\MEG\sample\sample_audvis_trunc-meg-eeg-oct-6-fwd.fif...
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    Distance information added...
    [done]
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    Distance information added...
    [done]
    2 source spaces read
    Desired named matrix (kind = 3523 (FIFF_MNE_FORWARD_SOLUTION_GRAD)) not available
    Read MEG forward solution (7928 sources, 306 channels, free orientations)
    Desired named matrix (kind = 3523 (FIFF_MNE_FORWARD_SOLUTION_GRAD)) not available
    Read EEG forward solution (7928 sources, 60 channels, free orientations)
    Forward solutions combined: MEG, EEG
    Source spaces transformed to the forward solution coordinate frame


In [26]:
mne.viz.plot_alignment(subject=None, trans=fwd["mri_head_t"], info=fwd["info"], src=fwd["src"], eeg=dict(original=0.2, projected=0.8))

Using outer_skin.surf for head surface.
Getting helmet for system 306m
Channel types::	grad: 203, mag: 102, eeg: 59
Projecting sensors to the head surface


In [27]:
   fwd["src"]._subject

'sample'

In [ ]:
info

In [17]:
fwd["mri_head_t"]

<Transform | MRI (surface RAS)->head>
[[ 0.99930954  0.00998479 -0.03578702 -0.00316745]
 [ 0.01275934  0.81240475  0.58295429  0.00685511]
 [ 0.0348942  -0.58300853  0.81171638  0.02888404]
 [ 0.          0.          0.          1.        ]]

In [12]:
plot_bem_kwargs = dict(
    subject="sample",
    subjects_dir=subjects_dir,
    brain_surfaces="white",
    orientation="coronal",
    src=src,

)

mne.viz.plot_bem(**plot_bem_kwargs)

Using surface: C:\Users\victor.ferat\mne_data\MNE-testing-data\subjects\sample\bem\inner_skull.surf
Using surface: C:\Users\victor.ferat\mne_data\MNE-testing-data\subjects\sample\bem\outer_skull.surf
Using surface: C:\Users\victor.ferat\mne_data\MNE-testing-data\subjects\sample\bem\outer_skin.surf


419MB [65:05:16, 1.79kB/s]


ValueError: Source space must be in MRI coordinates, got head

In [14]:
stc = mne.read_source_estimate(stc_fname)
stc

TypeError: 'SourceEstimate' object is not subscriptable